In [422]:
from pyplasm import *
import csv


immagine di riferimento
<img src="riferimento.jpg">

Pianta vista dall'alto
<img src="pianta5.png">

<img src="pianta3.png">

<img src="pianta4.png">

Funzione per generare la porta

In [423]:
def window(X,Y,Z,occupancy):
    def door0(dx,dy,dz):
        def door1(cod):
            
            scX = 0
            scY = 0
            scZ = 0
            for el in X:
                scX += el
            for el in Y:
                scY += el
            for el in Z:
                scZ += el
            scX = dx/scX
            scY = dy/scY
            scZ = dz/scZ
            
            structure = CUBOID([0,0,0])
            ix = 0
            sommaX = 0
            
            for piano in occupancy:
                page = CUBOID([0,0,0])
                iz = 0
                sommaZ = 0
                
                for riga in piano:
                    tmp = []
                    iy = 0
                    tuttiNegativi = 1

                    for val in riga:
                        if val == cod: 
                            tmp.append(Y[iy]*scY)
                            tuttiNegativi = 0
                        else: tmp.append(-Y[iy]*scY)

                        iy += 1

                    if tuttiNegativi == 0:
                        #1d
                        qy = QUOTE(tmp)
                        #2d
                        rxy = PROD([QUOTE([X[ix]*scX]), qy])
                        #ryz = PROD([qy, QUOTE([Z[iz]])])
                        #3d
                        row = PROD([rxy, QUOTE([Z[iz]*scZ])])
                        #unisco le righe
                        page = STRUCT([page, T(3)(sommaZ), row])

                    sommaZ += Z[iz]*scZ
                    iz += 1

                structure = STRUCT([structure, T(1)(sommaX), page])
                sommaX += X[ix]*scX
                ix += 1

            return structure
        
        cGlass = [210.0/255, 1.0, 1.0]
        cFrame = [80.0/255, 40.0/255, .0/255]
        frame = COLOR(cFrame)(door1(1))
        glass = COLOR(cGlass)(door1(2))
        
        return STRUCT([T([1,2])([-dx/2, -dy/2]),frame,glass])

    return door0

#ogni elemento delle righe di occ si riferisce alle y
occ1 = [[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1]]
occ2 = [[1,1,1,1,1],[1,0,1,0,1],[1,0,1,0,1],[1,0,1,0,1],[1,1,1,1,1]]
occ3 = [[1,1,1,1,1],[0,2,1,2,0],[0,2,1,2,0],[0,2,1,2,0],[1,1,1,1,1]]
occ4 = [[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1],[1,0,1,0,1],[1,1,1,1,1]]
occ = [occ1, occ2,occ3, occ4]


#VIEW(window([.03,.015,.015,.03],[.1,.3,.05,.3,.1],[.1,.5,.05,.5,.1],occ)(X, Y, Z)

In [424]:
j = 0
with open ('esterni.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 0:
                mura = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                j = 1
            else:
                mura = STRUCT([mura, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
mura = OFFSET([0.4, 0.4, 3])(mura)
#VIEW(mura)

In [425]:
j = 0
with open ('interni.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 0:
                muriInterni = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                j = 1
            else:
                muriInterni = STRUCT([muriInterni, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
muriInterni = OFFSET([.15, .15, 3])(muriInterni)
pianta = STRUCT([mura, T(2)(2.7), muriInterni])

In [426]:
j = 0
with open ('porte.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 0:
                porte = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                j = 1
            else:
                porte = STRUCT([porte, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
porte = OFFSET([.5, 1, 5])(porte)
porte = STRUCT([T(2)(-.05), porte])
pianta = DIFF([pianta, porte])
#VIEW(pianta)

In [427]:
j = 0
k = 0
with open ('finestre.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 1 and abs(punti[0] - punti[2]) < .05:
                finestreVerticali = STRUCT([finestreVerticali, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
            if j == 0 and abs(punti[0] - punti[2]) < .05:
                finestreVerticali = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                j = 1
            
            if k == 1 and abs(punti[1] - punti[3]) < .05:
                finestreOrizzontali = STRUCT([finestreOrizzontali, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
            if k == 0 and abs(punti[1] - punti[3]) < .05:
                finestreOrizzontali = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                k = 1
finestreVerticali = OFFSET([.8, .01, 1.5])(finestreVerticali)
finestreOrizzontali = OFFSET([.01, .8, 1.5])(finestreOrizzontali)
finestre = STRUCT([finestreVerticali, finestreOrizzontali])
finestre = STRUCT([T([1,2,3])([-.05, -.05, 1]), finestre])
#VIEW(STRUCT([pianta, finestre]))
pianta = DIFF([pianta, finestre])
#VIEW(pianta)

In [ ]:
j = 0
k = 0
with open ('porteInterne.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 1 and abs(punti[0] - punti[2]) < .05:
                porteInterneVerticali = STRUCT([porteInterneVerticali, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
            if j == 0 and abs(punti[0] - punti[2]) < .05:
                porteInterneVerticali = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                j = 1
            if k == 1 and abs(punti[1] - punti[3]) < .05:
                porteInterneOrizzontali = STRUCT([porteInterneOrizzontali, MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
            if k == 0 and abs(punti[1] - punti[3]) < .05:
                porteInterneOrizzontali = STRUCT([MKPOL([[[punti[0], punti[1], 0], [punti[2], punti[3], 0]],[[1,2]],[[1]]])])
                k = 1
porteInterneVerticali = OFFSET([.5, .01, 5])(porteInterneVerticali)
porteInterneOrizzontali = OFFSET([.01, .5, 5])(porteInterneOrizzontali)
porteInterne = STRUCT([porteInterneVerticali, porteInterneOrizzontali])
porteInterne = STRUCT([T(2)(-.03), porteInterne])
#VIEW(STRUCT([pianta, porteInterne]))
pianta = DIFF([pianta, porteInterne])
#VIEW(pianta)

In [ ]:
#correzione
a = [194.0, 537.7]
b = [239.3, 537.7]
c = [506.9, 535.5]
d = [552.0, 535.5]
A = CUBOID([(b[0] - a[0])/20 -.05, 2.0, 5.0])
C = CUBOID([(d[0] - c[0])/20 -.05, 2.0, 5.0])
AB = STRUCT([T([1,2])([a[0]/20 +.3, a[1]/20 -2]), A])
AB = STRUCT([AB, T([1,2])([c[0]/20 + .2, d[1]/20 -2]), C])
pianta  = DIFF([pianta, AB])

In [ ]:
# distinguo le finestre a seconda dell'orientamento
j = 0
k = 0
with open ('finestre.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            
            #finestre con lunghezza sull'asse x
            if j == 0 and abs(punti[1] - punti[3]) < .05:
                finestreX = STRUCT([T([1,2,3])([(punti[0]+punti[2])/2, punti[1],1]), R([1,2])(math.pi/2), window([.03,.015,.015,.03],[.1,.3,.05,.3,.1],[.1,.5,.05,.5,.1],occ)(.2, abs(punti[0]-punti[2])+.4, 1.5)])
                j = 1
                
            if j == 1 and abs(punti[1] - punti[3]) < .05:
                finestreX = STRUCT([finestreX, T([1,2,3])([(punti[0]+punti[2])/2, punti[1],1]), R([1,2])(math.pi/2), window([.03,.015,.015,.03],[.1,.3,.05,.3,.1],[.1,.5,.05,.5,.1],occ)(.2, abs(punti[0]-punti[2])+.6, 1.5)])
                
                #finestre con lunghezza sull'asse x
            if k == 0 and abs(punti[0] - punti[2]) < .05:
                finestreY = STRUCT([T([1,2,3])([punti[0], (punti[1]+punti[3])/2, 1]), window([.03,.015,.015,.03],[.1,.3,.05,.3,.1],[.1,.5,.05,.5,.1],occ)(.2, abs(punti[1]-punti[3])+.4, 1.5)])
                k = 1
                
            if k == 1 and abs(punti[0] - punti[2]) < .05:
                finestreY = STRUCT([finestreY, T([1,2,3])([punti[0], (punti[1]+punti[3])/2, 1]), window([.03,.015,.015,.03],[.1,.3,.05,.3,.1],[.1,.5,.05,.5,.1],occ)(.2, abs(punti[1]-punti[3])+.6, 1.5)])

finestre= STRUCT([T([1,2])([.05, .05]), finestreX, finestreY])
#VIEW(finestre)
pianta = STRUCT([pianta, finestre])
#VIEW(pianta)

In [ ]:
def portaEsterna(x, y):
    return STRUCT([COLOR([80.0/255, 40.0/255, .0/255])(CUBOID([abs(x)+.4, abs(y), 3.]))])

def portaEsternaV(x, y):
    return STRUCT([COLOR([80.0/255, 40.0/255, .0/255])(CUBOID([abs(x), abs(y)+.3, 3.]))])

j = 0
with open ('porte.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 0:
                if punti[0] > punti[2]:
                    porte = STRUCT([T([1,2])([(punti[2]), punti[1]]), portaEsterna(punti[0]-punti[2], .04)])
                else:
                    porte = STRUCT([T([1,2])([(punti[0]), punti[1]]), portaEsterna(punti[0]-punti[2], .04)])
                j = 1
            else:
                if punti[0] > punti[2]:
                    porte = STRUCT([porte, T([1,2])([(punti[2]), punti[1]]), portaEsterna(punti[0]-punti[2], .04)])
                else:
                    porte = STRUCT([porte, T([1,2])([(punti[0]), punti[1]]), portaEsterna(punti[0]-punti[2], .04)])
porte = STRUCT([T(1)(.05), porte])
pianta = STRUCT([pianta, porte])
#VIEW(pianta)

In [ ]:
j = 0
k = 0
with open ('porteInterne.lines', 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
        for row in spamreader:
            punti = []
            for val in row:
                punti.append(float(val)/20)
            if j == 1 and abs(punti[0] - punti[2]) < .05:
                if punti[1] > punti[3]:
                    porteV = STRUCT([porteV, T([1,2])([(punti[0]), punti[3]]), portaEsternaV(.02, punti[1]-punti[3])])
                else:
                    porteV = STRUCT([porteV, T([1,2])([(punti[0]), punti[1]]), portaEsternaV(.02, punti[1]-punti[3])])
            if j == 0 and abs(punti[0] - punti[2]) < .05:
                if punti[1] > punti[3]:
                    porteV = STRUCT([T([1,2])([(punti[0]), punti[3]]), portaEsternaV(.02, punti[1]-punti[3])])
                else:
                    porteV = STRUCT([T([1,2])([(punti[0]), punti[1]]), portaEsternaV(.02, punti[1]-punti[3])])
                j = 1
            if k == 1 and abs(punti[1] - punti[3]) < .05:
                if punti[0] > punti[2]:
                    porte = STRUCT([porte, T([1,2])([(punti[2]), punti[1]]), portaEsterna(punti[0]-punti[2], .02)])
                else:
                    porte = STRUCT([porte, T([1,2])([(punti[0]), punti[1]]), portaEsterna(punti[0]-punti[2], .02)])
            if k == 0 and abs(punti[1] - punti[3]) < .05:
                if punti[0] > punti[2]:
                    porte = STRUCT([T([1,2])([(punti[2]), punti[1]]), portaEsterna(punti[0]-punti[2], .02)])
                else:
                    porte = STRUCT([T([1,2])([(punti[0]), punti[1]]), portaEsterna(punti[0]-punti[2], .02)])
                k = 1
                
porteV = STRUCT([T(1)(.15), porteV])
porte = STRUCT([T(2)(.25), porte])
pianta = STRUCT([pianta, porte, porteV])
VIEW(pianta)